In [24]:
import json
import pandas as pd

# Load the unstructured JSON data
with open('/Users/yisheng/Desktop/Fetch_Analytics/users.json') as users:
    users_data = [json.loads(line) for line in users]

with open('/Users/yisheng/Desktop/Fetch_Analytics/brands.json') as brands:
    brands_data = [json.loads(line) for line in brands]

with open('/Users/yisheng/Desktop/Fetch_Analytics/receipts.json') as receipts:
    receipts_data = [json.loads(line) for line in receipts]

In [32]:
# Transform users.JSON data to flat structure
users = []
for user_record in users_data:
    users.append({
        'id': user_record['_id']['$oid'],
        'active': user_record.get('active', None),
        'created_date': pd.to_datetime(user_record['createdDate']['$date'], unit='ms') if 'createdDate' in user_record else None,
        'last_login_date': pd.to_datetime(record['lastLogin']['$date'], unit='ms') if 'lastLogin' in user_record else None,
        'role': user_record.get('role', None),
        'sign_up_source': user_record.get('signUpSource', None),
        'state': user_record.get('state', None)  # Handle missing 'state'
    })

# Transform brands.JSON data to flat structure, dimension table
brands = []
for brand_record in brands_data:
    brands.append({
        'id': brand_record['_id']['$oid'],
        'barcode': brand_record.get('barcode'),
        'category': brand_record.get('category'),
        'categoryCode': brand_record.get('categoryCode'),
        'cpg': brand_record['cpg']['$id']['$oid'],
        'ref': brand_record.get('ref'),
        'name': brand_record.get('name'),
        'topBrand': brand_record.get('topBrand')
    })

# Transform receipts.JSON to transactional data, fact table
receipts = []
for receipt_record in receipts_data:
    receipts.append({
        'id': brand_record['_id']['$oid'],
        'barcode': brand_record.get('barcode'),
        'category': brand_record.get('category'),
        'categoryCode': brand_record.get('categoryCode'),
        'cpg': brand_record['cpg']['$id']['$oid'],
        'ref': brand_record.get('ref'),
        'name': brand_record.get('name'),
        'topBrand': brand_record.get('topBrand')
    })


# Convert the structured data into a Pandas DataFrame
users_df = pd.DataFrame(users)
users_df.head()

brands_df = pd.DataFrame(brands)
brands_df.head()

receipts_df = pd.DataFrame(receipts)
receipts_df.head()

,id,barcode,category,categoryCode,cpg,ref,name,topBrand
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,None,test brand @1612366101024,False
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,None,Starbucks,False
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,None,test brand @1612366146176,False
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,None,test brand @1612366146051,False
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,None,test brand @1612366146827,False
